In [1]:
# do basic imports and unpack McMurdo data

#from pmagpy import ipmag
#reload(ipmag)
from pmagpy import pmag
from programs import new_builder as nb
from programs import data_model3
reload(data_model3)
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from programs.new_builder import Contribution

import pmagpy.controlled_vocabularies3 as cv



/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


got full_df
got parsed_df
got full_df
got parsed_df


In [2]:
dir_path = os.path.join(os.getcwd(), '3_0', 'Megiddo')
con = Contribution(dir_path)

loc_dm = con.tables['locations'].data_model.dm['locations']
loc_df = con.tables['locations'].df
site_dm = con.tables['sites'].data_model.dm['sites']
site_df = con.tables['sites'].df
samp_df = con.tables['samples'].df
samp_dm = con.tables['samples'].data_model.dm['samples']
spec_df = con.tables['specimens'].df
spec_dm = con.tables['specimens'].data_model.dm['specimens']
age_df = con.tables['ages'].df
age_dm = con.tables['ages'].data_model.dm['ages']
meas_df = con.tables['measurements'].df
meas_dm = con.tables['measurements'].data_model.dm['measurements']
cont_df = con.tables['contribution'].df
cont_dm = con.tables['contribution'].data_model.dm['contribution']
crit_df = con.tables['criteria'].df
crit_dm = con.tables['criteria'].data_model.dm['criteria']


current_con = con

# mess up some validations for locations
loc_df.loc['Tel Hazor', 'lat_s'] = 400.
loc_df['dir_inc'] = 5
loc_df.loc['Tel Hazor', 'lat_n'] = 'hello'
loc_df.loc[:, 'lithologies'] = ["Agate", "random"]
#current_con.tables.pop('sites')

# mess up some validations for sites
#site_df.pop('age')
#site_df['dir_tilt_correction'] = 1
#site_df['dir_tilt_correction'] = 'one'

# mess up some validations for ages
age_df.ix[1]['age'] = 'a string'
age_df.ix[1]['site'] = 'fake site'
age_df.ix[1]['age_low'] = 1000000000000.
age_df.pop('citations')

# mess up some validations for samples
samp_df.pop('citations')
samp_df.iloc[0].lon = 600.
samp_df.iloc[0].age = "another string"
samp_df.iloc[0].lat = "stringy"
samp_df.iloc[1].lat = 'hello'
samp_df.iloc[2].specimens = "hz05a2:fake"
samp_df.iloc[3].specimens = "fake : hz05a1"
samp_df.iloc[5].specimens = 'fake_specimen'
samp_df.iloc[7].site = 'fake_site'
samp_df.iloc[0].cooling_rate = 'a string'

# mess up some validations for measurements
meas_df.loc['mgh05a01:LP-PI-TRM1', 'magn_moment'] = 2
meas_df.loc['mgh05a01:LP-PI-TRM1', 'specimen'] = "fake_specimen"
meas_df.pop('experiment')

#current_df.head()
#current_df.head()
age_df.head()

-W- No such file: /Users/nebula/Python/PmagPy/3_0/Megiddo/images.txt


,age,age_high,age_low,age_unit,description,location,method_codes,site
0,-740,-732,-800,Years Cal AD (+/-),"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz05
1,a string,-732,1e+12,Years Cal AD (+/-),"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,fake site
2,-850,-800,-900,Years Cal AD (+/-),"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz07
3,-950,-900,-1000,Years Cal AD (+/-),"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz09
4,-950,-900,-1000,Years Cal AD (+/-),"""Tel-Hazor chronology. 2015 revision. Amnon Be...",Tel Hazor,GM-C14:GM-CC-ARCH,hz10


In [3]:
import pmagpy.controlled_vocabularies3 as cv
reload(cv)
vocab = cv.Vocabulary()
vocabulary, possible_vocabulary = vocab.get_controlled_vocabularies()

-I- Importing controlled vocabularies from https://earthref.org


In [4]:
## validation functions


# need to add requiredOneInGroup

def requiredUnless(col_name, df, arg, *args):
    arg_list = arg.split(",")
    arg_list = [arg.strip('"') for arg in arg_list]
    msg = ""
    for a in arg_list:
        if "." in a:
            continue
        if a not in df.columns:
            msg += "{} is required unless {} is present.  ".format(col_name, a)
    if msg:
        return msg
    else:
        return None
    return None


def requiredUnlessTable(col_name, df, arg, *args):
    """
    Col_name must be present in df unless
    arg (table_name) is present in contribution
    """
    table_name = arg
    if col_name in df.columns:
        return None
    elif table_name in current_con.tables:
        return None
    else:
        #print "{} is required unless table {} is present".format(col_name, table_name)
        return "{} is required unless table {} is present".format(col_name, table_name)

    
def requiredIfGroup(col_name, df, arg, *args):
    """
    Col_name is required if other columns of 
    the group arg are present.
    """
    group_name = arg
    groups = set()
    columns = df.columns
    for col in columns:
        if col not in current_dm.index:
            continue
        group = current_dm.loc[col]['group']
        groups.add(group)
    if group_name in groups:
        if col_name in columns:
            return None
        else:
            #print "{} is required if column group {} is used".format(col_name, group_name)
            return "{} is required if column group {} is used".format(col_name, group_name)
    return None


def required(col_name, df, arg):
    if col_name in df.columns:
        return None
    else:
        return "{} is required".format(col_name) 

def isIn(row, col_name, arg, dm, df):
    #grade = df.apply(func, args=(validation_name, arg, dm), axis=1)
    x = 0
    cell_value = row[col_name]
    if not cell_value:
        return None
    # if it's in another table
    cell_values = [v.strip(" ") for v in cell_value.split(":")]
    if "." in arg:
        table_name, table_col_name = arg.split(".")
        if table_name not in current_con.tables:
            return "Must contain a value from {} table. Missing {} table.".format(table_name, table_name)
        if table_col_name not in current_con.tables[table_name].df.columns:
            return "{} table is missing {} column, which is required for validation".format(table_name, table_col_name)
        possible_values = current_con.tables[table_name].df[table_col_name].unique()
        for value in cell_values:
            if value not in possible_values:
                return "This value: {} is not found in: {}".format(value, arg)
                break
    # if it's in the present table:
    else:
        possible_values = df[arg].unique()
        for value in cell_values:
            if value not in possible_values:
                return "This value: {} is not found in: {} column".format(value, arg)
                break
    return None
    
def checkMax(row, col_name, arg, *args):
    cell_value = row[col_name]
    if not cell_value:
        return None
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    #arg = float(arg)
    try:
        if float(cell_value) <= float(arg):
            return None
        else:
            #print "{} must be <= {}".format(str(cell_value), str(arg))
            return "{} must be <= {}".format(str(cell_value), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def checkMin(row, col_name, arg, *args):
    cell_value = row[col_name]
    if not cell_value:
        return None
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    try:
        if float(cell_value) >= float(arg):
            return None
        else:
            return "{} must be >= {}".format(str(cell_value), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def cv(row, col_name, arg, current_data_model, *args):
    cell_value = row[col_name]
    if not cell_value:
        return None
    if cell_value.lower() in [v.lower() for v in vocabulary[col_name]]:
        return None
    else:
        return "{} is not in {}".format(cell_value, arg)
        

# validate presence
presence_operations = {"required": required, "requiredUnless": requiredUnless,
                       "requiredIfGroup": requiredIfGroup, 
                       'requiredUnlessTable': requiredUnlessTable}
# validate values
value_operations = {"max": checkMax, "min": checkMin, "cv": cv, "in": isIn}

def split_func(string):
    """
    Take a string like 'requiredIf("arg_name")'
    return the function name and the argument:
    (requiredIf, arg_name)
    """
    ind = string.index("(")
    return string[:ind], string[ind+1:-1].strip('"')


def test_type(value, value_type):
    if not value:
        return None
    if value_type == "String":
        if str(value) == value:
            return None
        else:
            return "should be string"
    elif value_type == "Number":
        try:
            float(value)
            return None
        except ValueError:
            return "should be a number"
    elif value_type == "Integer":
        if isinstance(value, str):
            if str(int(value)) == value:
                return None
            else:
                return "should be an integer"
        else:
            if int(value) == value:
                return None
            else:
                return "should be an integer"
    else:
        return None
    #String, Number, Integer, List, Matrix, Dictionary, Text
    


def validate_df(df, dm):
    # check column validity
    cols = df.columns
    invalid_cols = [col for col in cols if col not in dm.index]
    for validation_name, validation in dm.iterrows():
        value_type = validation['type']
        if validation_name in df.columns:
            output = df[validation_name].apply(test_type, args=(value_type,))
            df["type_pass" + "_" + validation_name + "_" + value_type] = output

        val_list = validation['validations']
        if not val_list or isinstance(val_list, float):
            continue
        for num, val in enumerate(val_list):
            func_name, arg = split_func(val)
            if arg == "magic_table_column":
                continue
            # first validate for presence
            if func_name in presence_operations:
                func = presence_operations[func_name]
                grade = func(validation_name, current_df, arg)
                pass_col_name = "presence_pass_" + validation_name + "_" + func.__name__
                df[pass_col_name] = grade
    
            # then validate for correct values
            elif func_name in value_operations:
                func = value_operations[func_name]
                if validation_name in df.columns:
                    grade = df.apply(func, args=(validation_name, arg, dm, df), axis=1)
                    col_name = "value_pass_" + validation_name + "_" + func.__name__
                    if col_name in df.columns:
                        num_range = range(1, 10)
                        for num in num_range:
                            if (col_name + str(num)) in df.columns:
                                continue
                            else:
                                col_name = col_name + str(num)
                                break
                    df[col_name] = grade.astype(object)
    return df

  
    


    

# check that values pass validation
# validation checks to add:
# sv (suggested vocab)
# requiredOneInGroup
# requiredUnlessSynthetic


# re-do upload_magic to use contribution-level (??)

# first, do validations on each table in the contribution
# this will include removing unneeded data (RmKeys from old upload_magic)
# this will also include checking everything against the data model (strings are strings, etc.)g


# next, splat out each table into a file and wrap it up.  give it a sensible name.  

In [5]:
current_df = current_con.tables['sites'].df  
current_dm = current_con.tables['sites'].data_model.dm['sites']

current_df = validate_df(current_df, current_dm)

In [6]:
def get_validation_col_names(df):
    value_cols = df.columns.str.match("^value_pass_")
    present_cols = df.columns.str.match("^presence_pass")
    type_cols = df.columns.str.match("^type_pass_")

    value_col_names = df.columns[value_cols]
    present_col_names = df.columns[present_cols]
    type_col_names = df.columns[type_cols]

    validation_cols = np.where(value_cols, value_cols, present_cols)
    validation_cols = np.where(validation_cols, validation_cols, type_cols)
    validation_col_names = df.columns[validation_cols]
    return value_col_names, present_col_names, type_col_names, validation_col_names

value_col_names, present_col_names, type_col_names, validation_col_names = get_validation_col_names(current_df)
present_col_names

Index([u'presence_pass_age_requiredUnless',
       u'presence_pass_age_high_requiredUnless',
       u'presence_pass_age_low_requiredUnless',
       u'presence_pass_age_unit_required',
       u'presence_pass_aniso_tilt_correction_requiredIfGroup',
       u'presence_pass_aniso_type_requiredIfGroup',
       u'presence_pass_citations_required',
       u'presence_pass_dir_dec_requiredIfGroup',
       u'presence_pass_dir_inc_requiredIfGroup',
       u'presence_pass_dir_tilt_correction_requiredIfGroup',
       u'presence_pass_geologic_classes_required',
       u'presence_pass_geologic_types_required', u'presence_pass_lat_required',
       u'presence_pass_lithologies_required',
       u'presence_pass_location_required', u'presence_pass_lon_required',
       u'presence_pass_method_codes_required',
       u'presence_pass_paleo_lat_requiredIfGroup',
       u'presence_pass_paleo_lon_requiredIfGroup',
       u'presence_pass_result_quality_required',
       u'presence_pass_result_type_required', u'p

In [7]:
# incorrect data type problems
current_df[type_col_names].dropna(how='all', axis=1).head()

""
site
hz05
hz06
hz07
hz09
hz10


In [8]:
# missing column problems
current_df[present_col_names].dropna(how='all', axis=1).head()

,presence_pass_method_codes_required,presence_pass_result_quality_required,presence_pass_result_type_required
site,,,
hz05,method_codes is required,result_quality is required,result_type is required
hz06,method_codes is required,result_quality is required,result_type is required
hz07,method_codes is required,result_quality is required,result_type is required
hz09,method_codes is required,result_quality is required,result_type is required
hz10,method_codes is required,result_quality is required,result_type is required


In [9]:
# value problems:
current_df[value_col_names].dropna(how='all', axis=1).head()

,value_pass_age_high_checkMin,value_pass_age_low_checkMax
site,,
hz05,None,None
hz06,None,None
hz07,None,None
hz09,None,None
hz10,None,None


In [10]:
#  return all: 
#    rows with a problem
#    columns with a problem
#    cells with a problem
#    missing columns
#    extra columns



In [11]:
def extract_col_name(string):
    prefixes = ["presence_pass_", "value_pass_", "type_pass_"]
    end = string.rfind("_")
    for prefix in prefixes:
        if string.startswith(prefix):
            return prefix[:-6], string[len(prefix):end]
    return string, string


def check_row(row):
    ind = row[row.notnull()].index
    values = row[row.notnull()].values
    # to transformation with extract_col_name here???
    return dict(zip(ind, values))

#def check_row(row):
#    return True

def get_all_failures(df, value_cols, type_cols):
    print "problem rows (value & type problems)"
    df["num"] = range(len(df))
    # get column names for value & type validations
    names = value_cols.union(type_cols)
    # drop all non validation columns
    value_problems = df[names.union(["num"])]
    failing_items = value_problems.dropna(how="all", subset=names)
    if not len(failing_items):
        print "No problems"
        return
    failing_items = failing_items.dropna(how="all", axis=1)
    # get names of the failing items
    bad_items = list(failing_items.index)
    # get index numbers of the failing items
    bad_indices = list(failing_items["num"])
    zip(bad_indices, bad_items)
    #failing_items.drop("num", axis=1, inplace=True)#.apply(check_row, axis=1).values
    failing_items['issues'] = failing_items.drop("num", axis=1).apply(check_row, axis=1).values
    # maybe do a transformation in here so that you get "lon" instead of "value_pass_lon_checkMax"
    
    for ind, row in failing_items.iterrows():
        issues = row["issues"]
        print ind
        for key, issue in issues.items():
            issue_type, issue_col = extract_col_name(key)
            print "type: {:10}".format(issue_type), 
            print " | ",
            print "col name: {:10}".format(issue_col),
            print " | ",
            print "error message:", issue
            print "..."
        print '---'
        

current_df['num'] = range(len(current_df))
get_all_failures(current_df, value_col_names, type_col_names)

problem rows (value & type problems)
mgj06
type: value       |  col name: age_high    |  error message: -2800 must be >= -2600
...
type: value       |  col name: age_low     |  error message: -2600 must be <= -2800
...
---
mgk06
type: value       |  col name: age_high    |  error message: -1180 must be >= -1130
...
type: value       |  col name: age_low     |  error message: -1130 must be <= -1180
...
---


In [12]:

def get_bad_rows_and_cols(df, validation_names):
    df["num"] = range(len(df))
    problems = df[validation_col_names.union(["num"])]
    problems = problems.dropna(how='all', axis=0, subset=validation_names)
    problems = problems.dropna(how='all', axis=1)
    if not len(problems):
        return None, None, None
    bad_rows = zip(list(problems["num"]), list(problems.index))
    #
    bad_cols = problems.columns
    prefixes = ["value_pass_", "type_pass_"]
    missing_prefix = "presence_pass_"
    problem_cols = []
    missing_cols = []
    for col in bad_cols:
        pre, stripped_col = extract_col_name(col)
        print col
        for prefix in prefixes:
            if col.startswith(prefix):
                problem_cols.append(stripped_col)
                continue
        if col.startswith(missing_prefix):
            missing_cols.append(stripped_col)
    
    print 'Problem columns (wrong type or invalid value):'
    print problem_cols
    #
    print 'Missing columns:'
    print missing_cols
    return bad_rows, problem_cols, missing_cols
    
a, b, c = get_bad_rows_and_cols(current_df, validation_col_names)
if a:
    print "bad rows:", a[:10]
    print "problems:", b[:10]
    print "missing:", c[:10]

num
presence_pass_method_codes_required
presence_pass_result_quality_required
presence_pass_result_type_required
value_pass_age_high_checkMin
value_pass_age_low_checkMax
Problem columns (wrong type or invalid value):
[u'age_high', u'age_low']
Missing columns:
[u'method_codes', u'result_quality', u'result_type']
bad rows: [(0, 'hz05'), (1, 'hz06'), (2, 'hz07'), (3, 'hz09'), (4, 'hz10'), (5, 'hz11'), (6, 'hz13'), (7, 'hz15'), (8, 'hz18'), (9, 'hz20')]
problems: [u'age_high', u'age_low']
missing: [u'method_codes', u'result_quality', u'result_type']


In [13]:
## run through and validate entire contribution

for dtype in current_con.tables.keys()[1:]:
    print dtype
    current_df = current_con.tables[dtype].df
    current_dm = current_con.tables[dtype].data_model.dm[dtype]
    current_df = validate_df(current_df, current_dm)
    value_col_names, present_col_names, type_col_names, validation_col_names = get_validation_col_names(current_df)
    get_all_failures(current_df, value_col_names, type_col_names)
    get_bad_rows_and_cols(current_df, validation_col_names)

ages
problem rows (value & type problems)
1
type: type        |  col name: age         |  error message: should be a number
...
type: value       |  col name: site        |  error message: This value: fake site is not found in: sites.site
...
type: value       |  col name: age_high    |  error message: -732 must be >= 1e+12
...
type: value       |  col name: age_low     |  error message: 1e+12 must be <= -732
...
---
13
type: value       |  col name: site        |  error message: This value: mgh05 is not found in: sites.site
...
---
17
type: value       |  col name: site        |  error message: This value: mgh12t1 is not found in: sites.site
...
---
19
type: value       |  col name: age_high    |  error message: -2800 must be >= -2600
...
type: value       |  col name: age_low     |  error message: -2600 must be <= -2800
...
---
20
type: value       |  col name: age_high    |  error message: -1180 must be >= -1130
...
type: value       |  col name: age_low     |  error message: -1130 

## Filling in an existing dataframe

In [14]:
# keep all of df1, add in any extra from df2
df1 = pd.DataFrame(np.random.randint(1, 10, (3, 5)), columns=['one', 'two', 'three', 'four', 'five'])
df1.iloc[0, 1] = np.nan
df1.iloc[2, 2] = np.nan
df2 = pd.DataFrame(np.random.randint(1, 10, (3, 5)), columns=['one', 'three', 'five', 'seven', 'nine'])
df1

,one,two,three,four,five
0,3,NaN,8.0,9,2
1,7,6.0,7.0,2,6
2,6,6.0,NaN,8,6


In [15]:
df2

,one,three,five,seven,nine
0,1,5,6,1,5
1,5,5,6,9,7
2,8,3,5,4,2


In [16]:
unique_df2_cols = df2.columns.difference(df1.columns)
unique_df2 = df2[unique_df2_cols]

# this adds in all the unique columns that weren't in df1
concat_df = pd.concat([df1, unique_df2], axis=1)
# fills in null values in df1 with values from df2
concat_df.fillna(df2)

,one,two,three,four,five,nine,seven
0,3,NaN,8.0,9,2,5,1
1,7,6.0,7.0,2,6,7,9
2,6,6.0,3.0,8,6,2,4
